In [37]:
#optional width adjustment
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error

import matplotlib.pyplot as plt

In [3]:
# Template for repeated structure
result = {
    "results_arima": None,
    "results_sarima": None,
    "results_tft": None,
    "mape_arima": None,
    "mape_sarima":None,
    "mape_tft": None
}

# Main dictionary using the template
TSO_dict = {
    "50HZ": {
        "name": "50hz",
        **result
    },
    "Amprion": {
        "name": "amprion",
        **result
    },
    "Tennet": {
        "name": "tennet",
        **result
    },
    "TransnetBW": {
        "name": "transnet",
        **result
    }
}


In [31]:
# 
for i in TSO_dict:
  TSO_dict[i]['results_arima'] = pd.read_parquet(f"data/arima_results_test_{TSO_dict[i]['name']}.parquet")
  TSO_dict[i]['results_sarima'] = pd.read_parquet(f"data/sarima_results_test_{TSO_dict[i]['name']}_400_40024.parquet")
  TSO_dict[i]['results_tft'] = pd.read_parquet(f"data/tft_results_test_{TSO_dict[i]['name']}.parquet")
  TSO_dict[i]['mape_arima'] = mean_absolute_percentage_error(TSO_dict[i]['results_arima']['actual'],TSO_dict[i]['results_arima']['forecast'])
  TSO_dict[i]['mape_sarima'] = mean_absolute_percentage_error(TSO_dict[i]['results_sarima']['actual'],TSO_dict[i]['results_sarima']['forecast'])
  TSO_dict[i]['mape_tft'] = mean_absolute_percentage_error(TSO_dict[i]['results_tft']['target'],TSO_dict[i]['results_tft']['forecast'])
  #TSO_dict[i]['results_arima']['mape'] = mean_absolute_percentage_error(TSO_dict[i]['results_arima']['actual'],TSO_dict[i]['results_arima']['forecast'])
  #TSO_dict[i]['results_sarima']['mape'] = mean_absolute_percentage_error(TSO_dict[i]['results_arima']['actual'],TSO_dict[i]['results_arima']['forecast'])  
  #TSO_dict[i]['results_tft']['mape'] = mean_absolute_percentage_error(TSO_dict[i]['results_tft']['target'],TSO_dict[i]['results_tft']['forecast'])

## Performance Results of ARIMA, SARIMA and TFT for each TSO zone

In [17]:
for i in TSO_dict:
    print(f"MAPE {i:<10} ARIMA: {round(TSO_dict[i]['mape_arima'], 4):<15} SARIMA: {round(TSO_dict[i]['mape_sarima'], 4):<15} TFT: {round(TSO_dict[i]['mape_tft'], 4):<15}")


MAPE 50HZ       ARIMA: 0.2486          SARIMA: 0.1841          TFT: 0.2897         
MAPE Amprion    ARIMA: 0.1965          SARIMA: 0.1539          TFT: 0.2279         
MAPE Tennet     ARIMA: 0.3655          SARIMA: 0.2733          TFT: 0.3743         
MAPE TransnetBW ARIMA: 0.2604          SARIMA: 0.1885          TFT: 0.2303         


## Performance Results of ARIMA, SARIMA and TFT for each TSO zone divided by Summer and Winter 2023

In [24]:
import pandas as pd

data = []

# ARIMA Results
for i in TSO_dict:
    TSO_dict[i]["file"] = pd.read_parquet(f"data/arima_results_test_{TSO_dict[i]['name']}.parquet")
    TSO_dict[i]["mape"] = mean_absolute_percentage_error(TSO_dict[i]["file"]["actual"], TSO_dict[i]["file"]["forecast"])
    result_summer = TSO_dict[i]["file"].loc["2023-06":"2023-08"]
    result_winter = TSO_dict[i]["file"].loc["2023-12":"2024-02"]
    mape_summer = mean_absolute_percentage_error(result_summer["actual"], result_summer["forecast"])
    mape_winter = mean_absolute_percentage_error(result_winter["actual"], result_winter["forecast"])
    mape_improvement = (mape_winter - mape_summer) / mape_summer
    data.append({'Model': 'ARIMA', 'TSO': i, 'Overall MAPE': round(TSO_dict[i]["mape"], 4),
                 'MAPE Summer': round(mape_summer, 4), 'MAPE Winter': round(mape_winter, 4),
                 'Improvement': round(mape_improvement, 4)})

# SARIMA Results
for i in TSO_dict:
    result_summer = TSO_dict[i]['results_sarima'].loc["2023-06":"2023-08"]
    result_winter = TSO_dict[i]['results_sarima'].loc["2023-12":"2024-02"]
    mape_summer = mean_absolute_percentage_error(result_summer["actual"], result_summer["forecast"])
    mape_winter = mean_absolute_percentage_error(result_winter["actual"], result_winter["forecast"])
    mape_improvement = (mape_winter - mape_summer) / mape_summer
    data.append({'Model': 'SARIMA', 'TSO': i, 'Overall MAPE': round(TSO_dict[i]["mape"], 4),
                 'MAPE Summer': round(mape_summer, 4), 'MAPE Winter': round(mape_winter, 4),
                 'Improvement': round(mape_improvement, 4)})

# TFT Results
for i in TSO_dict:
    result_summer = TSO_dict[i]['results_tft'].loc["2023-06":"2023-08"]
    result_winter = TSO_dict[i]['results_tft'].loc["2023-12":"2024-02"]
    mape_summer = mean_absolute_percentage_error(result_summer["target"], result_summer["forecast"])
    mape_winter = mean_absolute_percentage_error(result_winter["target"], result_winter["forecast"])
    mape_improvement = (mape_winter - mape_summer) / mape_summer
    data.append({'Model': 'TFT', 'TSO': i, 'Overall MAPE': round(TSO_dict[i]["mape"], 4),
                 'MAPE Summer': round(mape_summer, 4), 'MAPE Winter': round(mape_winter, 4),
                 'Improvement': round(mape_improvement, 4)})


results_df = pd.DataFrame(data)

# Display the DataFrame
display(results_df)

,Model,TSO,Overall MAPE,MAPE Summer,MAPE Winter,Improvement
0,ARIMA,50HZ,0.2486,0.3038,0.1774,-0.4161
1,ARIMA,Amprion,0.1965,0.2126,0.1530,-0.2802
2,ARIMA,Tennet,0.3655,0.4248,0.2535,-0.4033
3,ARIMA,TransnetBW,0.2604,0.3065,0.1762,-0.4253
4,SARIMA,50HZ,0.2486,0.1773,0.1825,0.0294
5,SARIMA,Amprion,0.1965,0.1489,0.1589,0.0675
6,SARIMA,Tennet,0.3655,0.2519,0.2560,0.0162
7,SARIMA,TransnetBW,0.2604,0.1946,0.1699,-0.1269
8,TFT,50HZ,0.2486,0.2947,0.3433,0.1648
9,TFT,Amprion,0.1965,0.2396,0.2494,0.0409


---------